# Build and test the Inference container

This notebook will provide an step-by-step instruction to create a docker image for inference module of tile-based classification and test its performance.

> Note: Before proceeding, make sure to select the correct kernel. In the top-right corner of the notebook, choose the Jupyter kernel named `Bash`.


## Setup the environment

In [3]:
export WORKSPACE=/workspace/machine-learning-process
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}
printenv | grep RUNTIME
pwd

XDG_RUNTIME_DIR=/workspace/.local
RUNTIME=/workspace/machine-learning-process/runs
/workspace/machine-learning-process/runs


## Build the container

Inspect the container file:

In [4]:
cat ${WORKSPACE}/inference/make-inference/Dockerfile

# Stage 1: Build stage
FROM rockylinux:9.3-minimal AS build

# Install necessary build tools
RUN microdnf install -y curl tar

# Download the hatch tar.gz file from GitHub
RUN curl -L https://github.com/pypa/hatch/releases/download/hatch-v1.14.0/hatch-x86_64-unknown-linux-gnu.tar.gz -o /tmp/hatch-x86_64-unknown-linux-gnu.tar.gz

# Extract the hatch binary
RUN tar -xzf /tmp/hatch-x86_64-unknown-linux-gnu.tar.gz -C /tmp/

# Stage 2: Final stage
FROM rockylinux:9.3-minimal

# Set up a default user and home directory
ENV HOME=/home/neo

# Install essential libraries including expat and python3 without `config` commands
RUN microdnf install -y \
    expat \
    libpq \
    curl \
    git \
    wget \
    tar \
    && microdnf install -y python3 \
    && microdnf clean all

# Create a user with UID 1001, group root, and a home directory
RUN useradd -u 1001 -g 100 -m -d ${HOME} -s /sbin/nologin \
         -c "Default Neo User" neo && \
    mkdir -p /code /prod ${HOME}/.cache /home/neo/.local/

Build the container using `docker`:

In [5]:
docker build --format docker -t localhost/inference:latest ${WORKSPACE}/inference/make-inference


[1/2] STEP 1/4: FROM rockylinux:9.3-minimal AS build
Resolved "rockylinux" as an alias (/etc/containers/registries.conf.d/shortnames.conf)
Trying to pull docker.io/library/rockylinux:9.3-minimal...
Getting image source signatures
Copying blob 8ec988941d66 [--------------------------------------] 0.0b / 0.0b
Copying blob 8ec988941d66 skipped: already exists  
Copying blob 8ec988941d66 skipped: already exists  
Copying config dfaa211c6b [====================================] 578.0b / 578.0b
Copying config dfaa211c6b done  
Copying config dfaa211c6b done  
Writing manifest to image destination
Storing signatures
[1/2] STEP 2/4: RUN microdnf install -y curl tar
Package                                          Repository     Size
Installing:                                                         
 tar-2:1.34-7.el9.x86_64                         baseos     896.5 kB
Upgrading:                                                          
 curl-7.76.1-31.el9.x86_64                       baseos   

Show the `make-inference` help:

In [6]:
docker run --rm -it localhost/inference:latest hatch run prod:make-inference --help

Usage: make-inference [OPTIONS]

  A selected model with highest evaluation metrics will making an inference on
  a sentinel-2  data

Options:
  -i, --input_reference PATH  Url to sentinel-2 STAC Item to provide inference
                              on tif images for 12 common bands(excluding
                              cirrus)  [required]
  --help                      Show this message and exit.


## Test the normalized difference step in the container

Generate the normalized difference between the green and nir bands:

In [13]:
PRODUCT_ID="S2C_MSIL2A_20250420T093051_R136_T35ULS_20250420T125212"

In [14]:
docker run \
    -it \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/machine-learning-process/runs,target=/runs \
    --workdir=/runs \
    --user=1001:100 \
    --rm \
    localhost/inference:latest \
    hatch run make-inference \
    --input_reference https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/${PRODUCT_ID}


.   Creating environment: default0m
.   Checking dependencies
Asset href https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/35/U/LS/2025/04/20/S2C_MSIL2A_20250420T093051_N0511_R136_T35ULS_20250420T125212.SAFE/GRANULE/L2A_T35ULS_A003252_20250420T093552/IMG_DATA/R60m/T35ULS_20250420T093051_B01_60m.tif?st=2025-05-07T16%3A13%3A51Z&se=2025-05-08T16%3A58%3A51Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-05-08T01%3A31%3A47Z&ske=2025-05-15T01%3A31%3A47Z&sks=b&skv=2024-05-04&sig=lKHbkYcF8d1yP8QZDkK3EEFegJP6Q9DH8q/lwfBi6bw%3D with common name B01 found
Asset href https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/35/U/LS/2025/04/20/S2C_MSIL2A_20250420T093051_N0511_R136_T35ULS_20250420T125212.SAFE/GRANULE/L2A_T35ULS_A003252_20250420T093552/IMG_DATA/R10m/T35ULS_20250420T093051_B02_10m.tif?st=2025-05-07T16%3A13%3A51Z&se=2025-05-08T16%3A58%3A51Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9

List the outputs:

In [ ]:
tree ${RUNTIME}

## Clean-up 

In [ ]:
rm -fr ${RUNTIME}
docker rmi -f $(docker images -aq)